**Importing Libraries**

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

**Loading the dataset**

In [2]:
from google.colab import files
uploaded = files.upload()

Saving customer_churn.csv.csv to customer_churn.csv.csv


In [3]:
data = pd.read_csv('customer_churn.csv.csv')

**Dataset exploration**

In [5]:
print(data.head())
print(data.info())

   CustomerID  Tenure  MonthlyCharges  TotalCharges        Contract  \
0        1001       5            70.0         350.0  Month-to-month   
1        1002      10            85.5         850.5        Two year   
2        1003       3            55.3         165.9        One year   
3        1004       8            90.0         720.0  Month-to-month   
4        1005       2            65.2         130.4        One year   

      PaymentMethod  Churn  
0  Electronic check      1  
1      Mailed check      0  
2  Electronic check      1  
3       Credit card      0  
4  Electronic check      1  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   CustomerID      5 non-null      int64  
 1   Tenure          5 non-null      int64  
 2   MonthlyCharges  5 non-null      float64
 3   TotalCharges    5 non-null      float64
 4   Contract        5 non-nu

**Data Preprocessing**

In [8]:
data = data.dropna() #dropping rows with missing values
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   CustomerID      5 non-null      int64  
 1   Tenure          5 non-null      int64  
 2   MonthlyCharges  5 non-null      float64
 3   TotalCharges    5 non-null      float64
 4   Contract        5 non-null      object 
 5   PaymentMethod   5 non-null      object 
 6   Churn           5 non-null      int64  
dtypes: float64(2), int64(3), object(2)
memory usage: 412.0+ bytes


#Encoding categorical variables

In [10]:
data = pd.get_dummies(data, drop_first=True)
print(data.head())

   CustomerID  Tenure  MonthlyCharges  TotalCharges  Churn  Contract_One year  \
0        1001       5            70.0         350.0      1              False   
1        1002      10            85.5         850.5      0              False   
2        1003       3            55.3         165.9      1               True   
3        1004       8            90.0         720.0      0              False   
4        1005       2            65.2         130.4      1               True   

   Contract_Two year  PaymentMethod_Electronic check  \
0              False                            True   
1               True                           False   
2              False                            True   
3              False                           False   
4              False                            True   

   PaymentMethod_Mailed check  
0                       False  
1                        True  
2                       False  
3                       False  
4               

data splitting

In [13]:
from sklearn.model_selection import train_test_split

X = data.drop('Churn', axis=1)
y = data['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Developing and training the model

In [15]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model training

In [16]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, validation_data=(X_test,y_test))

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 1.0000 - loss: 1.2371e-06 - val_accuracy: 1.0000 - val_loss: 0.0997
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - accuracy: 0.5000 - loss: 21.1923 - val_accuracy: 0.0000e+00 - val_loss: 10.0250
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - accuracy: 0.7500 - loss: 13.4380 - val_accuracy: 0.0000e+00 - val_loss: 12.7697
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - accuracy: 0.5000 - loss: 46.3339 - val_accuracy: 0.0000e+00 - val_loss: 14.2565
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step - accuracy: 0.7500 - loss: 0.3438 - val_accuracy: 0.0000e+00 - val_loss: 15.5830
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - accuracy: 1.0000 - loss: 5.7324e-09 - val_accuracy: 0.0000e+00 - val_loss: 16.7133
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - accuracy: 0.7500 - loss: 41.6043 - val_accuracy: 0.0000e+00 - val_loss: 14.7237
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - accuracy: 0.7500 - loss: 14

model evaluation

In [17]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'Test accuracy: {test_acc}')

1/1 - 0s - 47ms/step - accuracy: 1.0000 - loss: 0.3503
Test accuracy: 1.0


model optimization

In [18]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

Saved artifact at '/tmp/tmpbvsp4pnh'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 8), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  135056728081808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135056728080656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135056728081040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135056731929488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135056728082384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135056731930064: TensorSpec(shape=(), dtype=tf.resource, name=None)


saving the model

In [20]:
model.save('churn_model.keras')